In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

#Notch filter


<h1>FFT Harmonic</h1>

In [163]:
def get_vertical_sin_image(img, A, amp, pha):
    """
    Get horizontal sine image for compensation.
    Faster than get_sine_img().
    """
    h, w = img.shape[:2]
    tmp = range(h)
    # create sine wave in 1 col
    tmp = np.array(range(h), np.float64)
    tmp = np.multiply(
        np.add(np.sin(np.add(np.multiply(tmp, A), pha)), 1), amp)
    tmp = tmp.tolist()
    sinimg = np.float32(np.array([tmp] * w)).transpose()

    return sinimg

@interact(f1=(0, 1e8), f2=(0, 1e8))
def notch_test(f1, f2):

    img = cv.imread('/Users/kolsha/Pictures/Kawasaki 250/Vv8igxLTBqI.jpg',0)

    rows, cols = img.shape
    crow, ccol = int(rows/2) , int(cols/2)     # center

    img_float32 = np.float32(img)

    #omega = 10000

    sin_v = get_vertical_sin_image(img, f1, 550, np.pi/2)

    sin_h = get_vertical_sin_image(img, f2, 550, np.pi/2)

    M = cv.getRotationMatrix2D((ccol,crow),90,2)
    sin_h = cv.warpAffine(sin_h,M,(cols,rows))

    img = img + sin_h + sin_v


    #sin = np.float32(sin)

    dft = cv.dft(img, flags = cv.DFT_COMPLEX_OUTPUT)
    dft_shift = dft 
    dft_shift = np.fft.fftshift(dft_shift)

    sin_v_dft = cv.dft(sin_v, flags = cv.DFT_COMPLEX_OUTPUT)
    sin_v_dft = np.fft.fftshift(sin_v_dft)

    sin_h_dft = cv.dft(sin_h, flags = cv.DFT_COMPLEX_OUTPUT)
    sin_h_dft = np.fft.fftshift(sin_h_dft)






    # create a mask first, center square is 1, remaining all zeros
    mask = np.zeros((rows, cols, 2), np.uint8)
    #mask[crow-30:crow+30, ccol-30:ccol+30] = 1
    mask[:,:] = 1

    sin_h_dft = np.abs(sin_h_dft)

    sin_v_dft = np.abs(sin_v_dft)

    print(sin_h_dft.min())
    mask[sin_h_dft >= omega] = 0

    mask[sin_v_dft >= omega] = 0





    # apply mask and inverse DFT
    fshift = dft_shift * mask

    #print(dft_shift.dtype, )

    #fshift = cv.mulSpectrums(dft_shift, sin_h_dft, flags=cv.DFT_ROWS)

    f_ishift = fshift

    f_ishift = np.fft.ifftshift(fshift)

    img_back = cv.idft(f_ishift, flags = cv.DFT_REAL_OUTPUT)

    #img_back = np.abs(img_back)


    #plt.subplot(121)
    plt.figure(figsize = (20,20))
    plt.imshow(img, cmap = 'gray', aspect='auto')
    plt.title('Input Image')
    plt.xticks([]), plt.yticks([])

    #plt.subplot(122)
    plt.figure(figsize = (20,20))
    plt.imshow(img_back, cmap = 'gray', aspect='auto')
    plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])

    plt.show()
    return(f1, f2)

interactive(children=(FloatSlider(value=50000000.0, description='f1', max=100000000.0), FloatSlider(value=5000…